<a href="https://colab.research.google.com/github/mesters-work/olist-analytics/blob/main/olist_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# =========================
# OLIST: FULL CLEAN PIPELINE
# =========================
# Expects these files under /content/olist_data/
#   - olist_orders_dataset.csv
#   - olist_order_items_dataset.csv
#   - olist_customers_dataset.csv
#   - olist_products_dataset.csv
#   - product_category_name_translation.csv

# Load raw datasets
orders    = pd.read_csv("/content/olist_data/olist_orders_dataset.csv")
items     = pd.read_csv("/content/olist_data/olist_order_items_dataset.csv")
customers = pd.read_csv("/content/olist_data/olist_customers_dataset.csv")
products  = pd.read_csv("/content/olist_data/olist_products_dataset.csv")
xlate     = pd.read_csv("/content/olist_data/product_category_name_translation.csv")

# -------------------------
# Order-level prep (delivery)
# -------------------------
orders["purchase_date"]  = pd.to_datetime(orders["order_purchase_timestamp"], errors="coerce")
orders["delivered_date"] = pd.to_datetime(orders["order_delivered_customer_date"], errors="coerce")
orders["estimated_date"] = pd.to_datetime(orders["order_estimated_delivery_date"], errors="coerce")

# Keep only delivered orders we can evaluate
orders = orders[orders["delivered_date"].notna()].copy()

# Delivery metrics
orders["delivery_days"] = (orders["delivered_date"] - orders["purchase_date"]).dt.days
orders["late_flag"]     = (orders["delivered_date"] > orders["estimated_date"]).astype(int)
orders["order_month"]   = orders["purchase_date"].dt.to_period("M").astype(str)

# -------------------------
# GMV at item level → orders
# -------------------------
items["gmv"] = items["price"] + items["freight_value"]
order_gmv = items.groupby("order_id", as_index=False)["gmv"].sum()

# Per-order table for Tableau (delivery + GMV)
orders_clean = (
    orders[["order_id","customer_id","purchase_date","delivered_date","estimated_date",
            "delivery_days","late_flag","order_month"]]
    .merge(order_gmv, on="order_id", how="left")
)
orders_clean.to_csv("/content/olist_orders_clean.csv", index=False)

# --------------------------------
# Cohorts: retention by signup month
# --------------------------------
orders["cohort_month"]  = orders.groupby("customer_id")["purchase_date"].transform("min").dt.to_period("M")
orders["order_month_p"] = orders["purchase_date"].dt.to_period("M")
orders["period_index"]  = (orders["order_month_p"] - orders["cohort_month"]).apply(lambda x: x.n)

cohort_data = (
    orders.groupby(["cohort_month","period_index"])["customer_id"]
          .nunique()
          .reset_index(name="active_customers")
)
cohort_sizes = (
    cohort_data[cohort_data["period_index"] == 0][["cohort_month","active_customers"]]
    .rename(columns={"active_customers":"cohort_size"})
)
cohort_data = cohort_data.merge(cohort_sizes, on="cohort_month", how="left")
cohort_data["retention_rate"] = cohort_data["active_customers"] / cohort_data["cohort_size"]
cohort_data["cohort_month"]   = cohort_data["cohort_month"].astype(str)

cohort_data.to_csv("/content/customer_cohorts.csv", index=False)

# -----------------------------------------
# Geo summary: % late deliveries by customer state
# -----------------------------------------
geo = orders.merge(customers[["customer_id","customer_state"]], on="customer_id", how="left")
geo_summary = (
    geo.groupby("customer_state", as_index=False)
       .agg(total_orders=("order_id","count"),
            late_orders =("late_flag","sum"))
)
geo_summary["pct_late"] = geo_summary["late_orders"] / geo_summary["total_orders"]

geo_summary.to_csv("/content/geo_deliveries.csv", index=False)

# ---------------------------------------------------
# Business mix: category & seller GMV (with translation)
# ---------------------------------------------------
# Attach English category names to products
products = products.merge(xlate, on="product_category_name", how="left")

# Join items → products to get English category labels
items_prod = items.merge(products[["product_id","product_category_name_english"]],
                         on="product_id", how="left")

# GMV by product category (English)
category_gmv = (
    items_prod.groupby("product_category_name_english", as_index=False)["gmv"]
              .sum()
              .sort_values("gmv", ascending=False)
)
category_gmv.to_csv("/content/category_gmv.csv", index=False)

# GMV by seller
seller_gmv = (
    items.groupby("seller_id", as_index=False)["gmv"]
         .sum()
         .sort_values("gmv", ascending=False)
)
seller_gmv.to_csv("/content/seller_gmv.csv", index=False)

print(
    "Exports written:",
    "\n- /content/olist_orders_clean.csv",
    "\n- /content/customer_cohorts.csv",
    "\n- /content/geo_deliveries.csv",
    "\n- /content/category_gmv.csv",
    "\n- /content/seller_gmv.csv",
)


Exports written: 
- /content/olist_orders_clean.csv 
- /content/customer_cohorts.csv 
- /content/geo_deliveries.csv 
- /content/category_gmv.csv 
- /content/seller_gmv.csv
